In [126]:
import os 
%pwd  #present working directory 

'C:\\Users\\ericz\\TextSummarizer'

In [127]:
new_directory = r'C:\Users\ericz\TextSummarizer'  # Change this to your desired path

os.chdir(new_directory)
%pwd

'C:\\Users\\ericz\\TextSummarizer'

### Basic Configuration

In [128]:
# Reading yaml files 
from dataclasses import dataclass
from pathlib import Path

# Creating fields to extract the zipfile, unzip the zipfile, and store it in some paths 
@dataclass # 
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path 

In [129]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories


### Configuration Updates


In [130]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH): #path initiated before ANY module 
        self.config = read_yaml(config_path) # read the YAML file path
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
# basic setup before starting any module 
    def get_data_ingestion_config(self) -> DataIngestionConfig: # read all the configurations specified in config.yaml 
        config=self.config.data_ingestion # reading data from data_ingestion in config.yaml 
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig( # return all values from DataIngestionConfig
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
    


In [131]:
import os
import urllib.request as request 
import zipfile 
from src.textSummarizer.logging import logger 


### Components

In [132]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 
    
    def download_file(self):  # Download the file 
        if not os.path.exists(self.config.local_data_file):  # If the file does not exist, retrieve it from the URL 
            try:
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file,
                )
                logger.info("File is downloaded successfully.")
            except Exception as e:
                logger.error(f"Error downloading the file: {e}")
        else:
            logger.info("File already exists.")
    
    def extract_zip_file(self):  # Unzip the file
        """
        Extracts the zip file into the data directory.
        Function returns None.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}.")
        except zipfile.BadZipFile:
            logger.error(f"Error: The file {self.config.local_data_file} is not a valid ZIP file.")
        except FileNotFoundError:
            logger.error(f"Error: The file {self.config.local_data_file} was not found.")
        except Exception as e:
            logger.error(f"An error occurred while extracting the zip file: {e}")

In [133]:
config=ConfigurationManager()
data_ingestion_config=config.get_data_ingestion_config()
data_ingestion=DataIngestion(config=data_ingestion_config)

data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2024-12-04 17:13:06,233: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-04 17:13:06,235: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-04 17:13:06,239: INFO: common: created directory at: artifacts]
[2024-12-04 17:13:06,242: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-04 17:13:07,377: INFO: 2241091367: File is downloaded successfully.]
[2024-12-04 17:13:07,485: INFO: 2241091367: Extracted zip file to artifacts/data_ingestion.]
